In [1]:
from ilgtfs import ExtendedGTFS


In [2]:
gtfs = ExtendedGTFS(r'data/gtfs_2016_05_25/israel-public-transportation.zip')

In [3]:
gtfs.load_stops()

Loading stops
27258 stops loaded


In [14]:
for s in gtfs.stops.values():
    print(s.nearest_train_station_id)
    break

37304


In [24]:
def export_stops_near_sations(gtfs, filename):
    with open(gtfs.at_path('stops_near_stations.txt'), 'w', encoding='utf8') as outf:
        outf.write('stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,parent_station_name,' +
                   'nearest_train_station_id,nearest_train_station_name,train_station_distance,routes_here,train_station_distance_category\n')
        for stop in gtfs.stops.values():
            if stop.parent_station != '': continue    # only present terminals ones
            if stop.train_station_distance > 800: continue
            if stop.train_station_distance > 500: cat = 'far'
            elif stop.train_station_distance > 300: cat = 'medium'
            elif stop.train_station_distance > 0: cat = 'near'
            else: cat  = 'train'
            line = ','.join([
                str(stop.stop_id),
                stop.stop_code,
                stop.stop_name,
                stop.stop_desc,
                str(stop.stop_lat),
                str(stop.stop_lon),
                stop.location_type,
                stop.parent_station,
                gtfs.stops[int(stop.parent_station)].stop_name if stop.parent_station != '' else '',
                str(stop.nearest_train_station_id),
                gtfs.stops[stop.nearest_train_station_id].stop_name,
                str(stop.train_station_distance),
                ' '.join(stop.routes_stopping_here),
                cat
            ])
            outf.write(line + '\n')

export_stops_near_sations(gtfs, 'stops_near_stations.txt')

In [21]:
def f(gtfs: ExtendedGTFS):
    gtfs.load_trips()
    gtfs.load_routes()

    trip_story_id_to_service = defaultdict(lambda: set())
    for trip in gtfs.trips.values():
        trip_story_id_to_service[trip.trip_story_id].add(trip.service)

    c = Counter()
    for service_set in trip_story_id_to_service.values():
        c[len(service_set)] += 1
    print(c)

    with open('data/tmp.csv', 'w') as f:
        for trip_story_id, services in trip_story_id_to_service.items():
            for service in services:
                fields = [trip_story_id, service.start_date, service.end_date, ' '.join(str(x) for x in service.days)]
                line = ','.join(str(x) for x in fields)
                f.write(line + '\n')

    with_2_stories = [route for route in gtfs.routes.values() if len(route.trip_stories) > 1]
    print("There are %d routes with more than one trip story" % len(with_2_stories))


